# ストラテジー
+ 前日の終値から大きく下げて始まった場合、前日の終値までギャップを埋める
+ ギャップは、20ポイント〜50ポイント
+ <font color=red>【？】</font>取引は寄り付き5分後
+ [Day Trading Strategy | A Simple Strategy to Day Trade The Markets: The Opening Gap](http://www.leadingtrader.com/10/day-trading-strategy-a-simple-strategy-to-day-trade-the-markets-the-opening-gap/)



In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor,RSI
from quantopian.pipeline.data import morningstar
from zipline import TradingAlgorithm  
from quantopian.pipeline.filters import Q1500US
from quantopian.pipeline.factors import AverageDollarVolume
import numpy as np
import talib 
import pandas as pd 

### ToDo

チェック|Todo|結果|メモ
---|---|---|---
|GapDownして最初の5分で⇑⇓に動く銘柄の特徴||
✓|前日の出来高と最初の一分の出来高を比較してその後の動き|[結果](https://www.evernote.com/view/notebook/122f9aca-9772-4953-80af-81b53134f10f?locale=ja#b=177d468e-bf46-408c-8f29-97655b654be5&st=p&n=122f9aca-9772-4953-80af-81b53134f10f)は良さそう|
|dollar_volume.percentile_betweenを色々変えてみる||
|GapDownの絶対値で評価．-1％以下等||
✓|rsi_under_60 = rsi < 60|不必要なフィルター|


In [ ]:

class Gap(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.close]
    window_length = 2
    def compute(self, today, assets, out, open_price, close):
        out[:] = open_price[-1] / close[-2] - 1 
        
class PrevClose(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 2
    def compute(self, today, assets, out, close):
        out[:] = close[-2]
        
class PrevVolume(CustomFactor):
    inputs = [USEquityPricing.volume]
    window_length = 2
    def compute(self, today, assets, out, close):
        out[:] = close[-2]
        
def make_pipeline():
    base_universe = Q1500US()
    yesterday_close = PrevClose()
    yesterday_volume = PrevVolume()
    today_open = USEquityPricing.open
    dollar_volume = AverageDollarVolume(window_length=30)
    # rsi = RSI()#default window_length = 15
    # rsi_under_60 = rsi < 60
    ## gap = today_open / yesterday_close - 1 では出来ない．
    ## TypeError: unsupported operand type(s) for /: 'BoundColumn' and 'BoundColumn'
    # gap = Gap()
    
    #ToDo この範囲を色々変えてみる．
    high_dollar_volume = dollar_volume.percentile_between(98, 100)
    pipe = Pipeline(
        
        columns = {
            'yesterday_close': yesterday_close,
            'yesterday_volume': yesterday_volume,
            'yesterday_turnover': yesterday_close * yesterday_volume,
#             'today_open': today_open.latest, 
            'dollar_volume': dollar_volume,
            'high_dollar_volume': high_dollar_volume, 
#             'gap': gap, 
#             'rsi': rsi, 
        },
        screen = base_universe & high_dollar_volume #& rsi_under_60
        
    )
    return pipe


def check_gdgu(context, data):
    context.output = pipeline_output('pipe')
    context.security_list = context.output.index
    
    # GUGD sid_list
    context.gapdowns = context.output.sort_values(by='gap').head(5).index
    context.gapups =  context.output.sort_values(by='gap').tail(5).index
    
def gapdown_minutes_data(results):
    '''
    results: run_pipelineの返り値
    '''
    d = dict() ## gapdown のその日の分足データ
    for date in results.index.get_level_values(0).unique():
        gap_downs = results.ix[date].sort_values(by='gap').head(5).index
        pan = get_pricing(gap_downs, start_date=date, end_date = date, frequency = 'minute',)
        pan.minor_axis = map(lambda x: x.symbol,pan.minor_axis)
        d[date.strftime("%Y-%m-%d")] = pan 

    return d

def gapdown_pipeline_data(results):
    '''
    results: run_pipelineの返り値
    '''
    d_pipe = dict() ## gapdown の pipeline データ
    for date in results.index.get_level_values(0).unique():
        gap_downs = results.ix[date].sort_values(by='gap').head(5).index
        pipe = results.ix[date].ix[gap_downs]
        pipe.index = map(lambda x: x.symbol,pipe.index)
        d_pipe[date.strftime("%Y-%m-%d")] = pipe
    return d_pipe


def my_plot(d, d_pipe, n=None):
    '''
    d: gapdown_minutes_data の返り値
    d_pipe: gapdown_pipeline_data の返り値
    n: float. 昨日のTurnoverに対して，今日の09：30-31のTurnoverは n%以上動いた
    '''

    return1 = list()
    return2 = list()
    df_high = pd.DataFrame() 
    df_low = pd.DataFrame()

    for k in sorted(d.keys()):
        open_price = d[k]['close_price'].ix[0]
        volume = d[k]['volume'].ix[0]
        yesterday_turnover = d_pipe[k]['yesterday_turnover']
        ## 今日の オープン時の turnover が，昨日のクローズのTurnoverのn％以上の銘柄だけ選択
        if n:
            shock_ratio = open_price * volume / yesterday_turnover > n 
            shock_stocks = shock_ratio[shock_ratio==True]

        else: 
            shock_stocks = open_price
        
        if not shock_stocks.empty:
            shock_stocks_sids = shock_stocks.index
            
            highs = list()
            lows = list()
            for i in range(1,61):
                high = d[k]['high'][shock_stocks_sids].fillna(method='pad', axis=1).ix[i]
                low = d[k]['low'][shock_stocks_sids].fillna(method='pad', axis=1).ix[i]
                highs.append((high/open_price-1).values)
                lows.append((low/open_price-1).values)
            return1.append(pd.DataFrame(highs))
            return2.append(pd.DataFrame(lows))
            
    if return1:
        df_high = pd.concat(return1, axis=1)
    if return2:        
        df_low = pd.concat(return2, axis=1)
    
    print("Done")
    
    return df_high, df_low 

results = run_pipeline(make_pipeline(), start_date='2015-1-1', end_date='2017-1-10')
d = gapdown_minutes_data(results)
d_pipe  = gapdown_pipeline_data(results)
#df_high, df_low  = my_plot(d, d_pipe, n=0.5) # n=0.03
        

In [ ]:
print(df_high)

#len(df_low.columns)

In [ ]:
nlist = [0.1, 0.05, 0.02, 0.01, None]
for n in nlist:
    df_high, df_low  = my_plot(d, d_pipe, n=n) # n=0.03

    if not df_high.empty:
        #df_high.T.dropna().describe().ix['50%'].plot()
        s = pd.Series(df_high.T.dropna().describe().ix['50%'], name = n)
        print df_high.T.dropna().describe().ix['count'][0]
        s.plot(legend=True)

In [ ]:
# pd.concat([df[0] for df in return1], axis=1).plot(style='o', legend=False, xlim=[-5, 65], ylim=[-0.10,0.10], marker='.')  

nlist = [0.1, 0.05, 0.02, 0.01, None]
for n in nlist:
    df_high, df_low  = my_plot(d, d_pipe, n=n) # n=0.03

    if not df_high.empty:
        #df_high.T.dropna().describe().ix['50%'].plot()
        s = pd.Series(df_high.T.dropna().describe().ix['mean'], name = n)
        s.plot(legend=True)
        

In [ ]:
#df_high.T.dropna().describe().ix['50%'].plot()
#df_low.T.dropna().describe().ix['50%'].plot()
df_high.dropna(axis=1)#.plot(style='o', legend=False, xlim=[-5, 65], ylim=[-0.10,0.10], marker='.')  
#df_high.dropna(axis=1).T.describe().ix['mean'].plot()
df_high.dropna(axis=1).T.describe().ix['50%'].plot()
df_low.dropna(axis=1).T.describe().ix['50%'].plot()

In [ ]:
pd.DataFrame(np.random.random_sample((3, 3)), columns=list('abc'))[:]

In [ ]:
pd.concat(l, axis=1).dropna().describe().ix["50%"].sort_values()[:-1].sum()#.plot(legend=False)
#pd.concat(l, axis=1).plot(legend=False)


In [ ]:
fundamentals = init_fundamentals()  
fundamental_df = get_fundamentals(  
query(  
fundamentals.financial_statement_filing.accession_number,  
fundamentals.financial_statement_filing.file_date,  
fundamentals.earnings_report.basic_eps,  
fundamentals.earnings_report.form_type_earnings_reports,  
fundamentals.earnings_report.normalized_basic_eps,  
fundamentals.financial_statement_filing.period_ending_date,  
)
.limit(20),
date, "1y"
)


In [ ]:
date
pd.to_datetime(fundamental_df.T['period_ending_date'],)#unit='s'



In [ ]:
bt = get_backtest("5913f1ce4bc63361fd781403")


In [ ]:
bt.create_full_tear_sheet()